In [1]:
from __future__ import print_function, division
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
%matplotlib inline

from halotools.sim_manager import CachedHaloCatalog, UserSuppliedHaloCatalog
from halotools.empirical_models import HodModelFactory

from halotools.empirical_models import TrivialPhaseSpace, ZuMandelbaum15Cens, ZuMandelbaum15Sats, \
                                        Leauthaud11Cens, Leauthaud11Sats, Zheng07Cens, Zheng07Sats, \
                                        NFWPhaseSpace, SubhaloPhaseSpace

In [2]:
from halotools.mock_observables import tpcf
from halotools.mock_observables.ia_correlations import ee_3d, ed_3d

In [3]:
from halotools.empirical_models.ia_models.ia_model_components import CentralAlignment, RandomAlignment, RadialSatelliteAlignment, SubhaloAlignment
from halotools.empirical_models.ia_models.ia_strength_models import RadialSatelliteAlignmentStrength

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Eliminate halos with 0 for halo_axisA_x(,y,z)
def mask_bad_halocat(halocat):
    bad_mask = (halocat.halo_table["halo_axisA_x"] == 0) & (halocat.halo_table["halo_axisA_y"] == 0) & (halocat.halo_table["halo_axisA_z"] == 0)
    bad_mask = bad_mask ^ np.ones(len(bad_mask), dtype=bool)
    halocat._halo_table = halocat.halo_table[ bad_mask ]

In [6]:
def create_model(cen_mu, sat_mu, selected_halocat):
    cens_occ_model = Zheng07Cens()
    cens_prof_model = TrivialPhaseSpace()
    cens_orientation = CentralAlignment(central_alignment_strength=cen_mu)
    sats_occ_model = Zheng07Sats()
    prof_args = ("satellites", np.logspace(10.5, 15.2, 15))
    sats_prof_model = SubhaloPhaseSpace(*prof_args)
    sats_prof_model = NFWPhaseSpace(conc_mass_model="dutton_maccio14")
    sats_orientation = RadialSatelliteAlignment(satellite_alignment_strength=sat_mu, halocat=selected_halocat)

    model_instance = HodModelFactory(centrals_occupation = cens_occ_model,
                                     centrals_profile = cens_prof_model,
                                     satellites_occupation = sats_occ_model,
                                     satellites_profile = sats_prof_model,
                                     centrals_orientation = cens_orientation,
                                     satellites_orientation = sats_orientation,
                                     model_feature_calling_sequence = (
                                     'centrals_occupation',
                                     'centrals_profile',
                                     'satellites_occupation',
                                     'satellites_profile',
                                     'centrals_orientation',
                                     'satellites_orientation')
                                    )
    
    return model_instance

In [7]:
def correlate(model, rbins, halocat):
    gal_table = model.mock.galaxy_table
    coords = np.array( [ gal_table["x"], gal_table["y"], gal_table["z"] ] ).T
    orientations = np.array( [ gal_table["galaxy_axisA_x"], gal_table["galaxy_axisA_y"], gal_table["galaxy_axisA_z"] ] ).T
    
    xi = tpcf(coords, rbins, period = halocat.Lbox)
    omega = ed_3d(coords, orientations, coords, rbins, period=halocat.Lbox)
    eta = ee_3d(coords, orientations, coords, orientations, rbins, period=halocat.Lbox)
    
    return xi, omega, eta

# Shared Parameters

In [8]:
seed=132358712
cen_mu = 1.0
sat_mu = 1.0

In [9]:
rbins = np.logspace(-1,1.2,21)
rbin_centers = (rbins[:-1]+rbins[1:])/2.0

# Cached Halocat (Bolplanck)

In [10]:
cached_halocat = CachedHaloCatalog(simname='bolplanck', halo_finder='rockstar', redshift=0, version_name='halotools_v0p4')
mask_bad_halocat(cached_halocat)

In [58]:
cached_model = create_model(cen_mu, sat_mu, cached_halocat)

cached_model.populate_mock(cached_halocat,seed=seed)

In [59]:
cached_xi, cached_omega, cached_eta = correlate(cached_model, rbins, cached_halocat)

# "User-Supplied" Halocat (Bolplanck)

In [80]:
selected_keys = ['halo_id', 'halo_upid', 'halo_mvir', 'halo_axisA_z','halo_axisA_x','halo_axisA_y', 'halo_vx','halo_vy','halo_vz', 'halo_x', 'halo_y', 'halo_z',
                'halo_mvir_host_halo', 'halo_hostid', 'halo_mpeak', 'halo_rvir', 'halo_tidal_force']
selected_keys = ['halo_id', 'halo_upid', 'halo_mvir', 'halo_axisA_z','halo_axisA_x','halo_axisA_y', 'halo_vx','halo_vy','halo_vz', 'halo_x', 'halo_y', 'halo_z',
                'halo_hostid', 'halo_rvir']

In [81]:
columns = { key : cached_halocat.halo_table[key] for key in selected_keys }

In [82]:
particle_mass = cached_halocat.particle_mass
redshift = cached_halocat.redshift
Lbox = cached_halocat.Lbox

In [83]:
user_halocat = UserSuppliedHaloCatalog(particle_mass=particle_mass, redshift=redshift, Lbox=Lbox, **columns)
mask_bad_halocat(user_halocat)

In [84]:
user_model = create_model(cen_mu, sat_mu, user_halocat)

user_model.populate_mock(user_halocat,seed=seed)

HalotoolsError: Your model requires that the ``halo_rvir`` key appear in the halo catalog,
but this column is not available in the catalog you attempted to populate.


In [44]:
user_halocat.halo_table

halo_id,halo_upid,halo_mvir,halo_axisA_z,halo_axisA_x,halo_axisA_y,halo_vx,halo_vy,halo_vz,halo_x,halo_y,halo_z,halo_mvir_host_halo,halo_hostid,halo_mpeak,halo_rvir,halo_tidal_force
int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32
2811042639,-1,200800000000000.0,19.2231,59.7891,-18.8001,16.1,8.51,-78.88,36.17984,43.14082,17.96339,200800000000000.0,2811042639,200800000000000.0,1.190447,0.11954
2811055606,-1,179600000000000.0,41.2062,34.6803,17.8882,2.46,264.77,-128.08,45.36644,49.54417,40.01593,179600000000000.0,2811055606,179600000000000.0,1.146849,0.50587
2809250167,-1,129800000000000.0,-17.5268,38.9596,24.3626,18.49,124.89,-35.19,22.02318,13.88261,9.80153,129800000000000.0,2809250167,129800000000000.0,1.029343,0.07568
2809483946,-1,103000000000000.0,24.7744,-10.3568,38.9949,-281.37,-115.39,-391.28,12.29788,36.67881,34.18085,103000000000000.0,2809483946,103000000000000.0,0.952978,0.09677
2809272603,-1,99470000000000.0,29.2183,52.7796,6.18836,-43.87,292.95,-171.47,10.66037,26.12877,22.5009,99470000000000.0,2809272603,99470000000000.0,0.941893,0.12465
2809261554,-1,95030000000000.0,1.58553,39.1732,-18.0791,281.14,229.38,224.59,1.93592,29.5494,18.1172,95030000000000.0,2809261554,99560000000000.0,0.92767197,0.18597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821890540,2821900623,1395000000.0,0.82819,3.56472,2.10328,317.78,6.4,116.3,226.41032,227.20032,219.2786,37230000000000.0,2821900623,46950000000.0,0.022712,2.62804
2821924303,2821924306,1395000000.0,-6.35833,6.06988,6.92538,254.4,84.66,-368.57,245.33875,231.06435,247.26727,3644000000000.0,2821924306,49430000000.0,0.022712,1.80501


In [ ]:
user_xi, user_omega, user_eta = correlate(user_model, rbins, user_halocat)

# Plot Comparisons

In [ ]:
# Create a 1x3 grid of subplots
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Plot data in the first subplot
axes[0].plot(rbin_centers, cached_xi, label="Cached Catalog")
axes[0].plot(rbin_centers, user_xi, "o", label="User Catalog")
axes[0].set_title(r'$\xi$')
axes[0].legend()

# Plot data in the second subplot
axes[1].plot(rbin_centers, cached_omega)
axes[1].plot(rbin_centers, user_omega, "o")
axes[1].set_title(r'$\omega$')

# Plot data in the third subplot
axes[2].plot(rbin_centers, cached_eta)
axes[2].plot(rbin_centers, user_eta, "o")
axes[2].set_title(r'$\eta$')

# Adjust spacing between subplots
plt.tight_layout()

# Display the plots
for ax in axes:
    ax.set_xscale("log")
    ax.set_yscale("log")
plt.show()

In [48]:
print(min(cached_halocat.halo_table["halo_mvir"]))
print(max(cached_halocat.halo_table["halo_mvir"]))
print(max(cached_halocat.halo_table["halo_mvir"])-min(cached_halocat.halo_table["halo_mvir"]))

619900000.0
1269000000000000.0
1268999400000000.0


In [53]:
cached_halocat.halo_table["halo_id", "halo_upid", "halo_mvir", "halo_mvir_host_halo"]

halo_id,halo_upid,halo_mvir,halo_mvir_host_halo
int64,int64,float32,float32
2811042639,-1,200800000000000.0,200800000000000.0
2811055606,-1,179600000000000.0,179600000000000.0
2809250167,-1,129800000000000.0,129800000000000.0
2809483946,-1,103000000000000.0,103000000000000.0
2809272603,-1,99470000000000.0,99470000000000.0
2809261554,-1,95030000000000.0,95030000000000.0
...,...,...,...
2821890540,2821900623,1395000000.0,37230000000000.0
2821924303,2821924306,1395000000.0,3644000000000.0
